### Esse script serve para extrair as informações presentes no site SteamDB de maneira automatizada e exportá-los a uma planilha Excel.
### Lembro que para executar o script, é necessário instalar as bibliotecas necessárias.
### Versão do Python: **3.12.4**.

In [ ]:
# Para instalar as bibliotecas é só tirar o cometário abaixo
# !pip install selenium seleniumbase pandas numpy pandas-gbq google-auth

### Importação das Bibliotecas

In [1]:
from seleniumbase import Driver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
import warnings
import pandas_gbq
from google.oauth2 import service_account

warnings.filterwarnings(action='ignore')

### WebScrapping do site da SteamDB 

In [ ]:
# Criando um webdriver
driver = Driver(uc=True)

# Url pra acessar o site do steamDB
url = 'https://steamdb.info/sales/?min_reviews=0'

# Passando pelo captcha
driver.uc_open_with_reconnect(url, 4)
driver.uc_gui_click_captcha()

# Timer pra carregar a página
time.sleep(10)

# Clicar no botão para mostrar todos os itens da página
driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[1]/div[1]/select/option[8]').click()

# Total de itens na página
itens = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[1]/div/div/div/div[1]/h2/span').text.replace('.', '')
print('Total de itens:', itens)

# Timer pra carregar todos os itens da página
time.sleep(20)

# Dicionário de listas para adicionar os dados
dados = {
    'Nome': [],
    'Data de Lançamento': [],
    'Avaliação Percentual': [],
    'Preço': [], 
    'Desconto Percentual': [],
    'Inicio do Desconto': [],
    'Fim do Desconto': []
}

# Loop para adicionar os dados de cada item
for item in range(1, int(itens) + 1):
    # Atribuindo os dados do item a cada variável relacionada
    nome = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[3]/a').text
    lancamento = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[7]').get_attribute('data-sort')                                   
    avaliacao = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[6]').get_attribute('data-sort')
    preco = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[5]').get_attribute('data-sort')
    desconto = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[4]').get_attribute('data-sort')
    inicio_desconto = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[9]').get_attribute('data-sort')
    fim_desconto = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[8]').get_attribute('data-sort')

    # Adição do valor das variáveis relacionadas do item
    dados['Nome'].append(nome)
    dados['Data de Lançamento'].append(lancamento)
    dados['Avaliação Percentual'].append(avaliacao)
    dados['Preço'].append(preco)
    dados['Desconto Percentual'].append(desconto)
    dados['Inicio do Desconto'].append(inicio_desconto)
    dados['Fim do Desconto'].append(fim_desconto)

    print(f'Dados do item {item}/{itens} adicionados com sucesso')

driver.close()

### Tratamento dos Dados

In [3]:
# Criando um Dataframe com os dados extraído do SteamDB
df_dados = pd.DataFrame(dados)
df_dados

,Nome,Data de Lançamento,Avaliação Percentual,Preço,Desconto Percentual,Inicio do Desconto,Fim do Desconto
0,Pistol Whip,1573084800,90.39,4449,50,1731088914,1731348000
1,Age of Wonders 4,1682985600,79.52,12999,35,1731002536,1731607200
2,ICARUS,1638489600,70.44,5099,50,1731002514,1731607200
3,Age of Water,1714003200,62.52,7450,50,1731002576,1731607200
4,Baldur's Gate 3,1691020800,95.98,15999,20,1730744570,1731348000
...,...,...,...,...,...,...,...
4711,Spirit Mancer,1732233600,-1,3149,10,1727946002,1732298400
4712,Bottle,1664323200,-1,782,10,1730744667,1731348000
4713,Grid Tales,1639440000,-1,1673,10,1730744634,1731348000
4714,My Wet Leto Comic-A Day of Green,1730332800,-1,2699,10,1730387402,1731607200


In [4]:
# Transformando a coluna de Avaliação para float e os valores -1 em NaN
df_dados['Avaliação Percentual'] = df_dados['Avaliação Percentual'].astype(float).replace(-1, np.nan)

# Transformando a coluna de preço em inteiro e dividindo por 100 para retratar os centavos
df_dados['Preço'] = df_dados['Preço'].astype(int) / 100

# Transformando a coluna de Desconto em inteiro
df_dados['Desconto Percentual'] = df_dados['Desconto Percentual'].astype(int)

# Transformando a coluna de Data de Lançamento de Timestamp para Datetime (Troquei Timestamps zerados por NaN para não gerar confusão)
df_dados['Data de Lançamento'] = df_dados['Data de Lançamento'].replace('0', np.nan)
df_dados['Data de Lançamento'] = pd.to_datetime(df_dados['Data de Lançamento'], unit='s')

# Transformando o Inicio de Desconto de Timestamp para Datetime (Troquei Timestamps zerados por NaN para não gerar confusão)
df_dados['Inicio do Desconto'] = df_dados['Inicio do Desconto'].replace('0', np.nan)
df_dados['Inicio do Desconto'] = pd.to_datetime(df_dados['Inicio do Desconto'], unit = 's')

# Transformando o Fim de Desconto de Timestamp para Datetime (Troquei Timestamps zerados por NaN para não gerar confusão) 
df_dados['Fim do Desconto'] = df_dados['Fim do Desconto'].replace('0', np.nan)
df_dados['Fim do Desconto'] = pd.to_datetime(df_dados['Fim do Desconto'], unit = 's')

In [5]:
# Dataset final com tratamento dos dados extraídos
df_dados

,Nome,Data de Lançamento,Avaliação Percentual,Preço,Desconto Percentual,Inicio do Desconto,Fim do Desconto
0,Pistol Whip,2019-11-07,90.39,44.49,50,2024-11-08 18:01:54,2024-11-11 18:00:00
1,Age of Wonders 4,2023-05-02,79.52,129.99,35,2024-11-07 18:02:16,2024-11-14 18:00:00
2,ICARUS,2021-12-03,70.44,50.99,50,2024-11-07 18:01:54,2024-11-14 18:00:00
3,Age of Water,2024-04-25,62.52,74.50,50,2024-11-07 18:02:56,2024-11-14 18:00:00
4,Baldur's Gate 3,2023-08-03,95.98,159.99,20,2024-11-04 18:22:50,2024-11-11 18:00:00
...,...,...,...,...,...,...,...
4711,Spirit Mancer,2024-11-22,NaN,31.49,10,2024-10-03 09:00:02,2024-11-22 18:00:00
4712,Bottle,2022-09-28,NaN,7.82,10,2024-11-04 18:24:27,2024-11-11 18:00:00
4713,Grid Tales,2021-12-14,NaN,16.73,10,2024-11-04 18:23:54,2024-11-11 18:00:00
4714,My Wet Leto Comic-A Day of Green,2024-10-31,NaN,26.99,10,2024-10-31 15:10:02,2024-11-14 18:00:00


### Passando o dataframe pandas para o GBQ 
### Lembrando que antes de rodar é necessário criar um projeto, dataset e tabela para preencher os valores abaixo. Os utilizados foram os que eu criei.

In [ ]:
# ID do projeto que criado no BQ
project_id = 'desafio-beanalytic' 

# ID da tabela criada no BQ
table_id = 'desafio-beanalytic.SteamDB_Extraction.Data'

# Acesso às credenciais do serviço da conta por meio do documento .json (Não coloquei pois é dado sensível)
credentials = service_account.Credentials.from_service_account_file('/caminho/do/seu/arquivo.json')

# Registrando o dataframe pra tabela BigQuery criada manualmente
pandas_gbq.to_gbq(df_dados, table_id, project_id, credentials=credentials)

100%|██████████| 1/1 [00:00<00:00, 1007.28it/s]


##### A partir do BQ, é possível exportar para uma planilha ou manipular os dados adicionados na tabela de lá.

In [6]:
# Dataframe no Excel se quiser
df_dados.to_excel('dados_steamDB.xlsx', index=0)